### Loading data

In [3]:
import pandas as pd
# download the csv file from here: https://www.google.com/covid19/mobility/index.html?hl=en
df = pd.read_csv('Global_Mobility_Report.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.sample(10)

,country_region_code,country_region,sub_region_1,sub_region_2,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
611814,US,United States,Tennessee,Unicoi County,NaN,47171.0,2020-07-10,NaN,1.0,NaN,NaN,-29.0,NaN
104563,HK,Hong Kong,NaN,NaN,NaN,NaN,2020-06-27,-13.0,3.0,-17.0,-18.0,-10.0,8.0
660216,US,United States,Virginia,King William County,NaN,51101.0,2020-03-13,14.0,25.0,NaN,NaN,-3.0,NaN
186200,NG,Nigeria,Cross River,NaN,NG-CR,NaN,2020-04-10,NaN,-26.0,-13.0,-29.0,-38.0,NaN
124179,IE,Ireland,County Offaly,NaN,IE-OY,NaN,2020-07-11,-32.0,7.0,205.0,NaN,-4.0,NaN
643496,US,United States,Texas,Wilson County,NaN,48493.0,2020-07-12,-20.0,-15.0,NaN,NaN,-20.0,NaN
240776,SA,Saudi Arabia,Riyadh Province,NaN,SA-01,NaN,2020-05-22,-44.0,7.0,-71.0,-83.0,-26.0,17.0
704942,VN,Vietnam,Hung Yen,NaN,VN-66,NaN,2020-04-01,-53.0,-40.0,-48.0,-59.0,-16.0,14.0
72968,FR,France,Brittany,NaN,FR-BRE,NaN,2020-05-20,-30.0,15.0,78.0,-32.0,-35.0,8.0
604683,US,United States,Tennessee,Hawkins County,NaN,47073.0,2020-06-20,11.0,22.0,NaN,NaN,0.0,NaN


### Cleaning steps

In [6]:
# We only focus on the trend in the U.S
df = df.loc[df['country_region']=="United States"]
# Drop the columns that we will not use for the analysis
df = df.drop(columns=['country_region_code','country_region','iso_3166_2_code', 'census_fips_code'])
# Rename the rest of the columns for easier accessing
df = df.rename(columns={'sub_region_1': 'state',
                        'sub_region_2': 'county',
                        'retail_and_recreation_percent_change_from_baseline': 'retail',
                        'grocery_and_pharmacy_percent_change_from_baseline': 'grocery',
                        'parks_percent_change_from_baseline': 'parks',
                        'transit_stations_percent_change_from_baseline': 'transit',
                        'workplaces_percent_change_from_baseline': 'workplaces',
                        'residential_percent_change_from_baseline': 'residential'})
# Drop the rows that contain any missing values from the state and county column
df = df.dropna(subset=['state', 'county'])

In [7]:
# convert the long format state name to short code in order to plot the map
state_codes = {
    'District of Columbia' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 
    'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 
    'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 
    'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 
    'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 
    'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 
    'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 
    'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI',
    'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 
    'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 
    'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 
    'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 
    'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 
    'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 
    'Nevada': 'NV', 'Maine': 'ME'}

df['state'] = df['state'].apply(lambda x : state_codes[x])

### Double check before exporting - everything looks fine

In [8]:
df.sample(10)

,state,county,date,retail,grocery,parks,transit,workplaces,residential
534771,NC,Halifax County,2020-02-25,11.0,-10.0,NaN,NaN,1.0,0.0
444609,MI,Allegan County,2020-06-14,35.0,33.0,NaN,NaN,-7.0,-2.0
619029,TX,Colorado County,2020-06-23,-5.0,NaN,NaN,5.0,-23.0,NaN
568376,OK,Payne County,2020-06-22,-15.0,0.0,NaN,NaN,-35.0,10.0
280231,AL,Cullman County,2020-07-03,-2.0,29.0,NaN,17.0,-53.0,13.0
664645,VA,Powhatan County,2020-03-26,-26.0,-34.0,NaN,NaN,-43.0,NaN
332127,GA,Bibb County,2020-03-02,13.0,7.0,-2.0,10.0,0.0,-1.0
575387,OR,Umatilla County,2020-07-18,-9.0,15.0,NaN,27.0,-8.0,0.0
644380,TX,Zapata County,2020-07-02,NaN,NaN,NaN,NaN,-37.0,NaN
670848,WA,Chelan County,2020-05-05,-12.0,9.0,94.0,-8.0,-37.0,11.0


### Exporting to the final csv file

In [9]:
df.to_csv('US_Mobility_Report.csv', index=False)

### Let's move on the COVID data

In [20]:
# Data can be downloaded from here: https://github.com/nytimes/covid-19-data
covid_df = pd.read_csv('us-states.csv')
covid_df.sample(10)

,date,state,fips,cases,deaths
7269,2020-07-12,Virginia,51,70670,1966
779,2020-03-16,Oklahoma,40,10,0
7980,2020-07-25,Texas,48,391609,5002
1088,2020-03-22,Michigan,26,1033,9
5050,2020-06-02,New Jersey,34,161545,11770
1566,2020-03-31,Hawaii,15,224,1
6232,2020-06-24,Arkansas,5,17375,240
1388,2020-03-27,Wyoming,56,74,0
3662,2020-05-08,Kentucky,21,6298,304
4488,2020-05-23,Louisiana,22,37165,2683


In [21]:
# convert the long format state name to short code in order to plot the map
covid_df['state'] = covid_df['state'].apply(lambda x : state_codes.get(x))
# remove the fips column
covid_df.drop(columns=['fips'], inplace=True)

In [22]:
covid_df.to_csv('covid_cases.csv', index=False)